[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/buildLittleWorlds/ml-math-with-densworld/blob/main/modules/04-applied-ml/notebooks/04-model-selection-capstone.ipynb)

# Lesson 4: Model Selection & Cross-Validation (Capstone)

*"The final day of testimony arrived. The Tribunal demanded a verdict: could my methods reliably identify forgeries? I proposed a test that would satisfy even the most skeptical judge—cross-validation. We would repeatedly hide some data, train on the rest, and measure performance on what was hidden. Only a model that consistently performed well across all such tests could be trusted."*  
— Mink Pavar, final day of the Great Forgery Trial

---

## The Final Test

The Great Forgery Trial of 912 reached its climax. Mink Pavar had demonstrated linear regression, explained the bias-variance trade-off, and shown how regularization could control complexity. But the Tribunal had one final question:

> *"How do we know your model will work on manuscripts we haven't examined yet? You've shown us training error and test error—but what if your test set was unusually easy or hard?"*

Mink Pavar smiled. This was the question he had been waiting for.

> *"We will use cross-validation—the most rigorous test of a model's true ability. We will train and test not once, but many times, each time holding out different data. The average performance across all these trials will tell us what to expect on truly unseen manuscripts."*

---

## Learning Objectives

By the end of this capstone lesson, you will:
1. Understand why a single train/test split is insufficient
2. Implement K-Fold Cross-Validation from scratch
3. Use cross-validation to select hyperparameters
4. Build a complete ML pipeline for the forgery detection task
5. Make principled decisions about model selection

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score, GridSearchCV

# Set random seed for reproducibility
np.random.seed(42)

# Nice plotting defaults
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Colab-ready data loading
BASE_URL = "https://raw.githubusercontent.com/buildLittleWorlds/ml-math-with-densworld/main/data/"

# Load all datasets
manuscripts = pd.read_csv(BASE_URL + "manuscript_features.csv")
expeditions = pd.read_csv(BASE_URL + "expedition_outcomes.csv")
creature_market = pd.read_csv(BASE_URL + "creature_market.csv")

print(f"Loaded {len(manuscripts)} manuscripts")
print(f"Loaded {len(expeditions)} expeditions")
print(f"Loaded {len(creature_market)} market transactions")

## Part 1: The Problem with a Single Split

*"A single test set is like a single witness. It may be truthful, but it may also be unrepresentative. What if, by chance, we happened to hide the easiest manuscripts? Or the hardest? Our confidence in the model would be misplaced."*  
— Mink Pavar

### Demonstrating Variability

In [ ]:
# Prepare manuscript data
feature_cols = ['stylometric_variance', 'era_marker_score', 'vocabulary_richness', 
                'avg_sentence_length', 'philosophical_term_density']

X = manuscripts[feature_cols].values
y = manuscripts['is_forgery'].astype(int).values

# Show how test error varies with different random splits
test_errors = []
for seed in range(50):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.3, random_state=seed
    )
    
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    model = Ridge(alpha=0.1)
    model.fit(X_train_scaled, y_train)
    
    y_pred = model.predict(X_test_scaled)
    mse = mean_squared_error(y_test, y_pred)
    test_errors.append(mse)

# Plot the variability
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.bar(range(50), test_errors, color='steelblue', alpha=0.7)
plt.axhline(np.mean(test_errors), color='red', linestyle='--', linewidth=2, 
            label=f'Mean: {np.mean(test_errors):.4f}')
plt.xlabel('Random Seed', fontsize=11)
plt.ylabel('Test MSE', fontsize=11)
plt.title('Test Error Varies with Random Split!', fontsize=12)
plt.legend()

plt.subplot(1, 2, 2)
plt.hist(test_errors, bins=15, color='coral', edgecolor='black', alpha=0.7)
plt.axvline(np.mean(test_errors), color='red', linestyle='--', linewidth=2, label=f'Mean: {np.mean(test_errors):.4f}')
plt.xlabel('Test MSE', fontsize=11)
plt.ylabel('Frequency', fontsize=11)
plt.title(f'Distribution of Test Errors\nStd: {np.std(test_errors):.4f}', fontsize=12)
plt.legend()

plt.tight_layout()
plt.show()

print(f"Test MSE range: {min(test_errors):.4f} to {max(test_errors):.4f}")
print(f"This {max(test_errors)/min(test_errors):.1f}x difference is just from random chance!")
print("\nA single split cannot give us reliable performance estimates.")

## Part 2: K-Fold Cross-Validation

*"Instead of one test, we perform K tests. We divide our data into K equal parts—called folds. Each fold takes a turn being the test set while the others train the model. The average across all K experiments is our estimate."*  
— Mink Pavar

### Visualizing K-Fold

In [ ]:
# Visualize 5-fold cross-validation
n_samples = 20  # Simplified for visualization
k_folds = 5

fig, ax = plt.subplots(figsize=(12, 4))

colors = ['steelblue', 'coral', 'green', 'purple', 'orange']
fold_size = n_samples // k_folds

for fold in range(k_folds):
    for sample in range(n_samples):
        sample_fold = sample // fold_size
        if sample_fold == fold:
            color = 'red'  # Test
            label = 'Test' if sample == fold * fold_size else None
        else:
            color = 'lightblue'  # Train
            label = 'Train' if sample == 0 and fold == 0 else None
        
        rect = plt.Rectangle((sample, k_folds - fold - 1), 0.9, 0.8, 
                              facecolor=color, edgecolor='black')
        ax.add_patch(rect)

ax.set_xlim(-0.5, n_samples + 0.5)
ax.set_ylim(-0.5, k_folds + 0.5)
ax.set_xlabel('Sample Index', fontsize=12)
ax.set_ylabel('Fold Number', fontsize=12)
ax.set_yticks(np.arange(k_folds) + 0.4)
ax.set_yticklabels([f'Fold {k_folds-i}' for i in range(k_folds)])
ax.set_title('5-Fold Cross-Validation\n(Red = Test, Blue = Train)', fontsize=13)

# Legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor='red', edgecolor='black', label='Test'),
                   Patch(facecolor='lightblue', edgecolor='black', label='Train')]
ax.legend(handles=legend_elements, loc='upper right')

plt.tight_layout()
plt.show()

print("Each row shows one fold's train/test split.")
print("Every sample gets to be in the test set exactly once.")
print("We train 5 models and average their performance.")

In [ ]:
# Implement K-Fold cross-validation from scratch
def manual_kfold_cv(X, y, model_class, model_params, k=5):
    """Perform K-fold cross-validation manually."""
    n_samples = len(X)
    fold_size = n_samples // k
    indices = np.arange(n_samples)
    np.random.shuffle(indices)
    
    fold_scores = []
    
    for fold in range(k):
        # Define test indices for this fold
        test_start = fold * fold_size
        test_end = test_start + fold_size if fold < k-1 else n_samples
        test_indices = indices[test_start:test_end]
        train_indices = np.concatenate([indices[:test_start], indices[test_end:]])
        
        # Split data
        X_train, X_test = X[train_indices], X[test_indices]
        y_train, y_test = y[train_indices], y[test_indices]
        
        # Scale features
        scaler = StandardScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train model
        model = model_class(**model_params)
        model.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = model.predict(X_test_scaled)
        mse = mean_squared_error(y_test, y_pred)
        fold_scores.append(mse)
        
        print(f"  Fold {fold+1}: MSE = {mse:.4f}")
    
    return fold_scores

print("Manual 5-Fold Cross-Validation for Forgery Detection:")
print("=" * 50)
scores = manual_kfold_cv(X, y, Ridge, {'alpha': 0.1})
print("-" * 50)
print(f"Mean MSE: {np.mean(scores):.4f} (+/- {np.std(scores):.4f})")

In [ ]:
# Compare to sklearn's implementation
from sklearn.pipeline import make_pipeline

# Create a pipeline (scaler + model)
pipeline = make_pipeline(StandardScaler(), Ridge(alpha=0.1))

# Perform cross-validation
cv_scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error')

print("\nSklearn 5-Fold Cross-Validation:")
print(f"MSE per fold: {-cv_scores}")
print(f"Mean MSE: {-cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

## Part 3: Using CV for Hyperparameter Selection

*"The Tribunal asked: 'How do you choose the regularization strength λ?' I explained: we use cross-validation to test each candidate value. The λ that gives the best average performance across all folds is our choice."*  
— Mink Pavar

In [ ]:
# Use CV to select optimal lambda for Ridge
lambdas = np.logspace(-4, 2, 20)
cv_results = []

for lam in lambdas:
    pipeline = make_pipeline(StandardScaler(), Ridge(alpha=lam))
    scores = cross_val_score(pipeline, X, y, cv=5, scoring='neg_mean_squared_error')
    cv_results.append({
        'lambda': lam,
        'mean_mse': -scores.mean(),
        'std_mse': scores.std()
    })

cv_df = pd.DataFrame(cv_results)

# Find best lambda
best_idx = cv_df['mean_mse'].idxmin()
best_lambda = cv_df.loc[best_idx, 'lambda']

# Plot
plt.figure(figsize=(10, 6))
plt.errorbar(cv_df['lambda'], cv_df['mean_mse'], yerr=cv_df['std_mse'], 
             fmt='o-', capsize=3, capthick=1, linewidth=2, markersize=6)
plt.axvline(best_lambda, color='green', linestyle='--', linewidth=2,
            label=f'Best λ = {best_lambda:.4f}')
plt.xscale('log')
plt.xlabel('λ (Regularization Strength)', fontsize=12)
plt.ylabel('Mean CV MSE (+/- std)', fontsize=12)
plt.title('Cross-Validation for Hyperparameter Selection', fontsize=13)
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

print(f"Optimal λ: {best_lambda:.4f}")
print(f"CV MSE at optimal λ: {cv_df.loc[best_idx, 'mean_mse']:.4f} (+/- {cv_df.loc[best_idx, 'std_mse']:.4f})")

## Part 4: Grid Search — Systematic Hyperparameter Tuning

*"When we have multiple hyperparameters, we search over a grid of all combinations. This is exhaustive but thorough."*  
— Mink Pavar

In [ ]:
# Grid search over multiple hyperparameters
from sklearn.linear_model import ElasticNet

# Define parameter grid
param_grid = {
    'elasticnet__alpha': [0.001, 0.01, 0.1, 1.0],
    'elasticnet__l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9]
}

# Create pipeline
pipeline = make_pipeline(
    StandardScaler(),
    ElasticNet(max_iter=10000)
)

# Perform grid search
grid_search = GridSearchCV(
    pipeline, 
    param_grid, 
    cv=5, 
    scoring='neg_mean_squared_error',
    return_train_score=True
)

grid_search.fit(X, y)

# Results
print("Grid Search Results:")
print("=" * 60)
print(f"Best parameters: {grid_search.best_params_}")
print(f"Best CV MSE: {-grid_search.best_score_:.4f}")

In [ ]:
# Visualize grid search results
results = pd.DataFrame(grid_search.cv_results_)

# Pivot for heatmap
alphas = param_grid['elasticnet__alpha']
l1_ratios = param_grid['elasticnet__l1_ratio']

scores_matrix = np.zeros((len(alphas), len(l1_ratios)))
for i, alpha in enumerate(alphas):
    for j, l1_ratio in enumerate(l1_ratios):
        mask = (results['param_elasticnet__alpha'] == alpha) & \
               (results['param_elasticnet__l1_ratio'] == l1_ratio)
        scores_matrix[i, j] = -results.loc[mask, 'mean_test_score'].values[0]

# Plot heatmap
plt.figure(figsize=(10, 6))
im = plt.imshow(scores_matrix, cmap='viridis_r', aspect='auto')
plt.colorbar(im, label='Mean CV MSE')

plt.xticks(range(len(l1_ratios)), [f'{r:.1f}' for r in l1_ratios])
plt.yticks(range(len(alphas)), [f'{a:.3f}' for a in alphas])
plt.xlabel('L1 Ratio (0=Ridge, 1=Lasso)', fontsize=12)
plt.ylabel('Alpha (Regularization Strength)', fontsize=12)
plt.title('Grid Search: Elastic Net Hyperparameter Tuning\n(Darker = Better)', fontsize=13)

# Mark best
best_alpha = grid_search.best_params_['elasticnet__alpha']
best_l1 = grid_search.best_params_['elasticnet__l1_ratio']
best_i = alphas.index(best_alpha)
best_j = l1_ratios.index(best_l1)
plt.plot(best_j, best_i, 'r*', markersize=20, label='Best')
plt.legend()

plt.tight_layout()
plt.show()

## Part 5: The Complete ML Pipeline — Forgery Detection System

*"Now we build the complete system. We will: (1) prepare the data, (2) select features, (3) choose the best model with cross-validation, and (4) evaluate on a truly held-out test set."*  
— Mink Pavar

### The Three-Way Split: Train / Validation / Test

In [ ]:
# IMPORTANT: Hold out a final test set BEFORE any model selection
# This test set must NEVER be used until the very end

# First split: separate final test set (20%)
X_trainval, X_test_final, y_trainval, y_test_final = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print("Data Split Strategy:")
print("=" * 50)
print(f"Train+Validation set: {len(X_trainval)} samples ({len(X_trainval)/len(X)*100:.0f}%)")
print(f"Final Test set: {len(X_test_final)} samples ({len(X_test_final)/len(X)*100:.0f}%)")
print(f"\nForgery rate in Train+Val: {y_trainval.mean()*100:.1f}%")
print(f"Forgery rate in Test: {y_test_final.mean()*100:.1f}%")
print("\n⚠️ The final test set will NOT be touched until the very end!")

In [ ]:
# Compare multiple models using cross-validation on train+val set
models_to_compare = {
    'Linear Regression': LinearRegression(),
    'Ridge (α=0.1)': Ridge(alpha=0.1),
    'Ridge (α=1.0)': Ridge(alpha=1.0),
    'Lasso (α=0.01)': Lasso(alpha=0.01, max_iter=10000),
    'Lasso (α=0.1)': Lasso(alpha=0.1, max_iter=10000),
    'Elastic Net': ElasticNet(alpha=0.05, l1_ratio=0.5, max_iter=10000),
}

cv_comparison = []
print("Model Comparison (5-Fold CV on Train+Val Set):")
print("=" * 60)

for name, model in models_to_compare.items():
    pipeline = make_pipeline(StandardScaler(), model)
    scores = cross_val_score(pipeline, X_trainval, y_trainval, 
                             cv=5, scoring='neg_mean_squared_error')
    cv_comparison.append({
        'Model': name,
        'Mean MSE': -scores.mean(),
        'Std MSE': scores.std()
    })
    print(f"{name:25}: MSE = {-scores.mean():.4f} (+/- {scores.std():.4f})")

cv_comparison_df = pd.DataFrame(cv_comparison)
best_model_name = cv_comparison_df.loc[cv_comparison_df['Mean MSE'].idxmin(), 'Model']
print(f"\n🏆 Best Model: {best_model_name}")

In [ ]:
# Visualize model comparison
plt.figure(figsize=(12, 6))
x_pos = np.arange(len(cv_comparison_df))

bars = plt.bar(x_pos, cv_comparison_df['Mean MSE'], 
               yerr=cv_comparison_df['Std MSE'], 
               capsize=5, color='steelblue', alpha=0.8, edgecolor='black')

# Highlight best
best_idx = cv_comparison_df['Mean MSE'].idxmin()
bars[best_idx].set_color('green')

plt.xticks(x_pos, cv_comparison_df['Model'], rotation=45, ha='right')
plt.ylabel('Mean CV MSE (+/- std)', fontsize=12)
plt.title('Model Comparison for Forgery Detection\n(Green = Best)', fontsize=13)
plt.tight_layout()
plt.show()

In [ ]:
# Final model selection and training
# Use the best hyperparameters found through CV

# Fine-tune the best model type
param_grid_final = {
    'ridge__alpha': np.logspace(-3, 1, 20)
}

pipeline_final = make_pipeline(StandardScaler(), Ridge())

grid_search_final = GridSearchCV(
    pipeline_final, 
    param_grid_final, 
    cv=5, 
    scoring='neg_mean_squared_error'
)

grid_search_final.fit(X_trainval, y_trainval)

print("Fine-tuned Ridge Regression:")
print(f"Best α: {grid_search_final.best_params_['ridge__alpha']:.4f}")
print(f"Best CV MSE: {-grid_search_final.best_score_:.4f}")

## Part 6: Final Evaluation on Held-Out Test Set

*"The moment of truth has arrived. We have selected our model without ever looking at the final test set. Now, and only now, do we evaluate on these truly unseen manuscripts."*  
— Mink Pavar

In [ ]:
# FINAL EVALUATION - This is the moment of truth!
# Train on ALL of train+val, evaluate on final test

best_model = grid_search_final.best_estimator_

# Make predictions on final test set
y_pred_final = best_model.predict(X_test_final)

# For classification metrics, threshold at 0.5
y_pred_class = (y_pred_final > 0.5).astype(int)

# Calculate metrics
final_mse = mean_squared_error(y_test_final, y_pred_final)
final_accuracy = accuracy_score(y_test_final, y_pred_class)

print("🎯 FINAL TEST SET EVALUATION 🎯")
print("=" * 60)
print(f"\nModel: Ridge Regression (α = {grid_search_final.best_params_['ridge__alpha']:.4f})")
print(f"\nFinal Test MSE: {final_mse:.4f}")
print(f"Final Test Accuracy: {final_accuracy*100:.1f}%")
print(f"\nCV MSE (estimate): {-grid_search_final.best_score_:.4f}")
print(f"Test MSE (actual): {final_mse:.4f}")
print(f"\nDifference: {abs(final_mse - (-grid_search_final.best_score_)):.4f}")
print("\n✅ The CV estimate was close to the true test performance!")

In [ ]:
# Detailed classification report
print("\nClassification Report:")
print("=" * 60)
print(classification_report(y_test_final, y_pred_class, 
                           target_names=['Authentic', 'Forgery']))

In [ ]:
# Confusion matrix visualization
cm = confusion_matrix(y_test_final, y_pred_class)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Confusion matrix
im = axes[0].imshow(cm, cmap='Blues')
axes[0].set_xticks([0, 1])
axes[0].set_yticks([0, 1])
axes[0].set_xticklabels(['Authentic', 'Forgery'])
axes[0].set_yticklabels(['Authentic', 'Forgery'])
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('Actual', fontsize=12)
axes[0].set_title('Confusion Matrix', fontsize=13)

# Add text annotations
for i in range(2):
    for j in range(2):
        text = axes[0].text(j, i, cm[i, j], ha='center', va='center', 
                           fontsize=20, color='white' if cm[i, j] > cm.max()/2 else 'black')

# Predictions vs actual
axes[1].scatter(y_test_final, y_pred_final, alpha=0.5, c='steelblue', s=60)
axes[1].axhline(0.5, color='red', linestyle='--', label='Decision boundary')
axes[1].axvline(0.5, color='gray', linestyle=':')
axes[1].set_xlabel('Actual (0=Authentic, 1=Forgery)', fontsize=12)
axes[1].set_ylabel('Predicted Probability', fontsize=12)
axes[1].set_title('Predictions vs Actual', fontsize=13)
axes[1].legend()

plt.tight_layout()
plt.show()

## Part 7: Feature Importance Analysis

*"The Tribunal wished to understand: which features are most important for detecting forgeries? The model's weights reveal this."*  
— Mink Pavar

In [ ]:
# Extract feature importance from final model
# Note: weights are for scaled features

# Get the Ridge model from the pipeline
scaler = best_model.named_steps['standardscaler']
ridge = best_model.named_steps['ridge']

# Weights for scaled features
weights = ridge.coef_

# Create importance dataframe
importance_df = pd.DataFrame({
    'Feature': feature_cols,
    'Weight': weights,
    'Abs_Weight': np.abs(weights)
}).sort_values('Abs_Weight', ascending=False)

print("Feature Importance for Forgery Detection:")
print("=" * 50)
print(importance_df.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 6))
colors = ['green' if w > 0 else 'red' for w in importance_df['Weight']]
plt.barh(importance_df['Feature'], importance_df['Weight'], color=colors, alpha=0.8, edgecolor='black')
plt.axvline(0, color='black', linewidth=0.5)
plt.xlabel('Weight (Positive = More Forgery-Like)', fontsize=12)
plt.title('Feature Weights in Forgery Detection Model', fontsize=13)
plt.tight_layout()
plt.show()

print("\nInterpretation:")
print("- Positive weight: Higher values increase forgery probability")
print("- Negative weight: Higher values decrease forgery probability")
print(f"\nMost important feature: {importance_df.iloc[0]['Feature']}")

---

## Exercises

### Exercise 1: Different K Values

Compare 3-fold, 5-fold, and 10-fold cross-validation. How does K affect the estimate variance?

In [ ]:
# Exercise 1: Compare different K values

k_values = [3, 5, 10]

# YOUR CODE HERE
# for k in k_values:
#     pipeline = make_pipeline(StandardScaler(), Ridge(alpha=0.1))
#     scores = cross_val_score(pipeline, X_trainval, y_trainval, cv=k, scoring='neg_mean_squared_error')
#     print(f"K={k}: Mean MSE = {-scores.mean():.4f} (+/- {scores.std():.4f})")

# Question: What happens to the variance as K increases?

### Exercise 2: Leave-One-Out Cross-Validation

Implement LOOCV where K = n (number of samples). Compare to 5-fold CV.

In [ ]:
# Exercise 2: Leave-One-Out CV
from sklearn.model_selection import LeaveOneOut

# YOUR CODE HERE
# loo = LeaveOneOut()
# pipeline = make_pipeline(StandardScaler(), Ridge(alpha=0.1))
# scores = cross_val_score(pipeline, X_trainval, y_trainval, cv=loo, scoring='neg_mean_squared_error')
# print(f"LOOCV: Mean MSE = {-scores.mean():.4f}")

### Exercise 3: Stratified K-Fold

When classes are imbalanced, use Stratified K-Fold to ensure each fold has similar class proportions.

In [ ]:
# Exercise 3: Stratified K-Fold
from sklearn.model_selection import StratifiedKFold

# YOUR CODE HERE
# Check class balance in each fold with regular vs stratified
# skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
# for train_idx, val_idx in skf.split(X_trainval, y_trainval):
#     print(f"Fold forgery rate: {y_trainval[val_idx].mean()*100:.1f}%")

### Exercise 4: Nested Cross-Validation

For truly unbiased model selection, use nested CV: inner loop for hyperparameter tuning, outer loop for performance estimation.

In [ ]:
# Exercise 4: Nested Cross-Validation

# YOUR CODE HERE
# Outer loop: 5-fold CV for performance estimation
# Inner loop: GridSearchCV for hyperparameter tuning
#
# outer_cv = KFold(n_splits=5, shuffle=True, random_state=42)
# inner_cv = KFold(n_splits=3, shuffle=True, random_state=42)
#
# nested_scores = []
# for train_idx, test_idx in outer_cv.split(X_trainval):
#     X_train_fold = X_trainval[train_idx]
#     y_train_fold = y_trainval[train_idx]
#     X_test_fold = X_trainval[test_idx]
#     y_test_fold = y_trainval[test_idx]
#     
#     # Inner CV for hyperparameter tuning
#     grid = GridSearchCV(pipeline, param_grid, cv=inner_cv)
#     grid.fit(X_train_fold, y_train_fold)
#     
#     # Evaluate on outer test fold
#     score = mean_squared_error(y_test_fold, grid.predict(X_test_fold))
#     nested_scores.append(score)

### Exercise 5: Build a Complete Pipeline for Expedition Success

Apply everything you've learned to predict expedition success from the expedition_outcomes data.

In [ ]:
# Exercise 5: Complete pipeline for expedition success prediction

# Features to use
exp_features = ['days_in_field', 'crew_size', 'leader_experience_years', 
                'creature_encounters', 'equipment_condition', 'morale_at_return']

# Target: success (binary)

# YOUR CODE HERE
# 1. Prepare data
# 2. Split into train+val and final test
# 3. Compare models with CV
# 4. Tune hyperparameters
# 5. Final evaluation

---

## Summary: The Complete ML Workflow

| Step | Purpose | Key Tool |
|------|---------|----------|
| 1. Data Split | Reserve final test set | `train_test_split` |
| 2. Preprocessing | Scale features | `StandardScaler` |
| 3. Model Comparison | Find best model type | `cross_val_score` |
| 4. Hyperparameter Tuning | Optimize parameters | `GridSearchCV` |
| 5. Final Evaluation | Unbiased performance | Held-out test set |

---

## Key Takeaways

1. **A single train/test split is unreliable** — results vary with random chance

2. **Cross-validation gives robust estimates** — average performance across K experiments

3. **Use CV for model selection AND hyperparameter tuning** — GridSearchCV combines both

4. **Hold out a final test set** — never touch it until the very end

5. **The CV estimate should match final test performance** — if not, something is wrong

6. **Feature importance reveals what matters** — interpret your model's weights

---

## The Verdict

*"The Tribunal deliberated. They had seen the evidence: linear regression derived from first principles, the bias-variance trade-off explained, regularization demonstrated, and cross-validation proving the model's reliability. On the final day, they rendered their verdict:*

*'Mink Pavar's mathematical methods have merit. The system he proposes—trained on known manuscripts, validated through cross-validation, and tested on held-out samples—provides a principled way to identify forgeries. We accept this approach as evidence in the Archives.'*

*The Great Forgery Trial of 912 was concluded. The forger was revealed to be a junior archivist who had studied under Grigsu Haldo himself—close enough to imitate the master's style, but not perfectly. The stylometric variance and era markers had betrayed him.*

*Mink Pavar returned to his studies, but his methods lived on. To this day, the Capital Archives uses mathematical classification to authenticate manuscripts. And scholars speak of the Forgery Trial as the moment when numbers became evidence."*

---

## Congratulations!

You have completed **Module 4: Applied ML** and the entire **ML Math with Densworld** course.

You now understand:
- **Statistics & Probability** — uncertainty, distributions, hypothesis testing
- **Linear Algebra** — vectors, matrices, transformations
- **Calculus** — derivatives, gradients, optimization
- **Applied ML** — regression, regularization, model selection

These foundations will serve you well as you explore more advanced topics: neural networks, deep learning, and beyond.

*"The mathematics of learning is now yours. Use it wisely."*  
— Mink Pavar